In [1]:
import os
import sys

sys.path.insert(0, os.path.abspath('/home/max/uva/mds_python'))

from mds.models import *
from mds.models.compact.user import UserCompactView
from mds.database.mongo import GetConfig
from mds.database.minio import GetMinioConfig

In [2]:
from mds.models.user import User
from mds.models.dataset import Dataset
from mds.models.download import Download
from mds.models.software import Software
from mds.models.computation import Computation

In [3]:
mongo_client = GetConfig()
mongo_database = mongo_client["test"]
mongo_collection = mongo_database["test_col"]

In [4]:
minio_client = GetMinioConfig()

In [5]:
test_user = User(**{
		"@id": "ark:99999/test-user1",
		"name": "Test User1",
		"type": "Person",
		"email": "testuser1@example.org",
		"password": "test1",
		"organizations": [],
		"projects": [],
		"datasets": [],
		"software": [],
		"computations": [],
		"evidencegraphs": []
		})

In [6]:
create_user_status = test_user.create(mongo_collection)

In [8]:
print(create_user_status)

Success: True	Message: 	StatusCode: 200


In [9]:
test_dataset = Dataset(**{
	"@id": "ark:99999/test-dataset",
	"@type": "evi:Dataset",
	"name": "test dataset",
	"owner": {
		"@id": "ark:99999/test-user1",
		"@type": "Person",
		"name": "Test User1",
		"email": "testuser1@example.org"
		}
    })

In [10]:
create_dataset = test_dataset.create(mongo_collection)

In [11]:
print(str(create_dataset))

Success: True	Message: 	StatusCode: 201


In [11]:
test_dataset.id

'ark:99999/test-dataset'

In [12]:
test_data_download = Download(**{
	"@id": "ark:99999/test-download",
	"name": "test-data.csv",
	"@type": "DataDownload",
	"encodingFormat": ".txt",
	"encodesCreativeWork": test_dataset.id,
    })

In [13]:
with open("tests/test-data.csv", "rb") as file:
    create_data_download = test_data_download.register(file, mongo_collection, minio_client)

In [14]:
str(create_data_download)

'Success: True\tMessage: \tStatusCode: 201'

In [15]:
# upload software
test_software = Software(**{
    "@id": "ark:99999/test-software",
    "name": "script1",
    "@type": "Software",
    "owner": {
		"@id": "ark:99999/test-user1",
		"@type": "Person",
		"name": "Test User1",
		"email": "testuser1@example.org"
		}
})

create_software_metadata = test_software.create(mongo_collection)

In [16]:
print(str(create_software_metadata))

Success: True	Message: 	StatusCode: 201


In [17]:
# upload script
test_software_object = Download(**{
    "@id": "ark:99999/test-software-script",
    "name": "test-script.py",
    "@type": "DataDownload",
    "owner": {
		"@id": "ark:99999/test-user1",
		"@type": "Person",
		"name": "Test User1",
		"email": "testuser1@example.org"
		},
    "encodingFormat": ".py",
    "encodesCreativeWork": test_software.id
})

In [18]:
with open("tests/test-script.py", "rb") as file:
    print(test_software_object.register(file, mongo_collection, minio_client))

Success: True	Message: 	StatusCode: 201


In [22]:
test_computation  =  Computation(**{
        "@id": "ark:99999/test-comp",
        "name":"test computation",
        "owner": {
        "@id": "ark:99999/test-user1",
        "@type": "Person",
        "name": "Test User1",
        "email": "testuser1@example.org"
        },
        "container": "python3",
        "usedSoftware":test_software.id ,
        "usedDataset": test_dataset.id
})

In [24]:
test_computation.json(by_alias=True)

'{"@id": "ark:99999/test-comp", "@type": "evi:Computation", "name": "test computation", "owner": {"@id": "ark:99999/test-user1", "@type": "Person", "name": "Test User1", "email": "testuser1@example.org"}, "dateCreated": null, "dateFinished": null, "associatedWith": [], "container": "python3", "usedSoftware": "ark:99999/test-software", "usedDataset": "ark:99999/test-dataset", "containerId": null, "@context": {"@vocab": "https://schema.org/", "evi": "https://w3id.org/EVI#"}}'

In [20]:
print(test_computation.create(mongo_collection))

Success: True	Message: 	StatusCode: 201


In [ ]:
test_computation.run_custom_container()

In [ ]:
register_computation(test_computation.containerId)